In [3]:
import seaborn as sns

In [6]:
df = sns.load_dataset('tips')

In [7]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [8]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [10]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['time'] = encoder.fit_transform(df['time'])

In [11]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [13]:
df['time'].value_counts()

0    176
1     68
Name: time, dtype: int64

In [15]:
X = df.drop('time', axis=1)

In [16]:
y = df['time']

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [18]:
from sklearn.impute import SimpleImputer ## Handelling Mising values
from sklearn.preprocessing import OneHotEncoder ## Handling Categorical values
from sklearn.preprocessing import StandardScaler ## Feature scalling
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer ## Automating the entire process

In [19]:
categorical_cols = ['sex', 'smoker', 'day']
numerical_cols = ['total_bill', 'tip', 'size']

In [20]:
## Feature engineering automation

numerical_pipeline = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)


categorical_pipeline = Pipeline(
    steps= [
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('One_Hot_Encoder', OneHotEncoder())
    ]
)

In [23]:
preprocessor = ColumnTransformer([
    ('nume_pipeline', numerical_pipeline, numerical_cols),
    ('cate_pipeline', categorical_pipeline, categorical_cols)
])

In [24]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [26]:
## Automate Model Training Process
models={
    'Random Forest':RandomForestClassifier(),
    'Decision Tree':DecisionTreeClassifier(),
    'SVC':SVC()

}

In [27]:
from sklearn.metrics import accuracy_score

In [30]:
def evaluate_model(X_train, y_train, X_test, y_test, models):
    
    report = {}
    
    for i in range(len(models)):
        model = list(models.values())[i]
        
        model.fit(X_train, y_train)
        
        y_test_pred = model.predict(X_test)
        
        test_model_score = accuracy_score(y_test, y_test_pred)
        
        report[list(models.keys())[i]] = test_model_score
        
    return report

In [31]:
evaluate_model(X_train,y_train,X_test,y_test,models)

{'Random Forest': 0.972972972972973,
 'Decision Tree': 0.9459459459459459,
 'SVC': 0.972972972972973}

In [32]:
classifier=RandomForestClassifier()

In [33]:
## Hypeparameter Tuning
params={'max_depth':[3,5,10,None],
              'n_estimators':[100,200,300],
               'criterion':['gini','entropy']
              }

In [34]:
from sklearn.model_selection import RandomizedSearchCV

In [35]:
cv=RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=3, n_estimators=200;, score=0.971 total time=   0.3s
[CV 2/5] END criterion=entropy, max_depth=3, n_estimators=200;, score=1.000 total time=   0.3s
[CV 3/5] END criterion=entropy, max_depth=3, n_estimators=200;, score=0.971 total time=   0.3s
[CV 4/5] END criterion=entropy, max_depth=3, n_estimators=200;, score=0.912 total time=   0.3s
[CV 5/5] END criterion=entropy, max_depth=3, n_estimators=200;, score=0.912 total time=   0.3s
[CV 1/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=0.941 total time=   0.5s
[CV 2/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=0.941 total time=   0.5s
[CV 3/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=1.000 total time=   0.5s
[CV 4/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=0.912 total time=   0.5s
[CV 5/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=0.912 tot

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [36]:
cv.best_params_

{'n_estimators': 200, 'max_depth': 5, 'criterion': 'entropy'}